In [2]:
import numpy as np
from datascience import *
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn import *
from sklearn.model_selection import train_test_split

from datetime import datetime, timezone
import missingno as msno
import seaborn as sns
import math
import statistics
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore





In [18]:
importedData = pd.read_csv("Participant_1058_Cleaned.csv")
importedDataLog = pd.read_csv("VR Timestamps for Phase B _ D_W_SP20 (1).csv")
importedData2 = pd.read_csv("1058 unix.csv")
MAINDATA = pd.read_csv("1058 food buffet data.csv")

C:\Users\Viraj\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (33,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def fNIRSdataProcessingPipeline(data, unixTS, hertz):
    data["1"] =(data["1"]/hertz)+1583859960
    return data


In [4]:
def logCustomPipeline(csvFile, tsCols):
    columns = csvFile.columns
    startingTime = csvFile.iloc[:, 1]
    for i in range(1, tsCols-1):
            csvFile[columns[i]] -= startingTime
    return csvFile

In [5]:
def pipeline3(csvFile):
    csvFileBase = csvFile._get_value(0, '1')
    csvFile["1"] = csvFile["1"]-csvFileBase
    return csvFile

In [6]:
def normalizePipeline(csvFile):
    rows = len(csvFile)
    result = csvFile
    columns = csvFile.columns
    columns = columns[1:len(columns)-1]
    for i in range(rows):
        rowNum = i
        currentRow = []
        for j in columns:
            currentRow.append(csvFile._get_value(i, j))
        mean = statistics.mean(currentRow)
        std = statistics.pstdev(currentRow)
        for h in range(len(columns)):
            result.at[rowNum, columns[h]] = (result._get_value(rowNum, columns[h]) - mean)/std
        
    return result

In [7]:
def normalizePipelineRows(csvFile):
    
    
    result = csvFile
    result = result.drop(["Unnamed: 33"], axis=1)

    scaler = StandardScaler()
    scaler.fit(result)

    df_normalized = pd.DataFrame(scaler.transform(result), columns=result.columns)
    df_normalized = df_normalized - df_normalized.mean()

        
    
    return df_normalized

In [8]:
def normalizePipelineCols(csvFile):
    
    
    result = csvFile
    result = result.drop(["Unnamed: 33"], axis=1)
    for i in range(len(result["1"])):
        result.iloc[i] = zscore(result.iloc[i])
    
    return result

In [9]:
r = fNIRSdataProcessingPipeline(importedData, 1583859960, 50)
L = logCustomPipeline(importedDataLog, 12)
z = pipeline3(importedData2)

In [10]:
xyz = normalizePipelineRows(MAINDATA)

In [11]:
xyz

avb = xyz['1']

nd = zscore(avb)

df_normalized = xyz.apply(zscore)

statistics.mean(df_normalized["1"])

2.3668184683246648e-17

In [12]:
zxy = normalizePipelineCols(MAINDATA)
statistics.mean(zxy.iloc[1])

2.2078298785159363e-18

In [13]:
np.std(zxy.iloc[1])

1.0

In [14]:
# xyz.to_csv('1058 food buffet data normalized.csv', index=False)
hhh

NameError: name 'hhh' is not defined

In [ ]:
xyz["2"].max

In [ ]:
abc.columns

In [20]:
MAINDATA

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,Unnamed: 33
0,0.00,28.002052,19.817099,47.819149,8.184955,24.096737,5.224597,29.321344,18.872190,24.006893,...,-54.685371,35.370872,24.114763,59.485634,11.256109,119.871323,-145.734436,-25.863215,265.605774,NaN
1,0.02,28.009739,19.821497,47.831234,8.188242,24.118839,5.223903,29.342752,18.894987,24.029949,...,-54.675510,35.399704,24.104576,59.504280,11.295129,119.885742,-145.739532,-25.853891,265.625275,NaN
2,0.04,28.010694,19.835388,47.846081,8.175305,24.140942,5.223208,29.364161,18.917784,24.067425,...,-54.649429,35.422760,24.117773,59.540535,11.304989,119.879013,-145.730041,-25.851129,265.609039,NaN
3,0.06,28.032797,19.834694,47.867489,8.198102,24.156313,5.232006,29.388330,18.924356,24.097212,...,-54.623344,35.445820,24.130970,59.576790,11.314849,119.901108,-145.730728,-25.829722,265.631836,NaN
4,0.08,28.047213,19.829601,47.876812,8.217612,24.178415,5.231312,29.409737,18.947153,24.112583,...,-54.597259,35.490025,24.129582,59.619606,11.360442,119.915527,-145.735825,-25.820398,265.651367,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4664,93.28,27.849163,20.063209,47.912373,7.785953,24.717382,5.553139,30.270533,19.164293,24.544863,...,-53.939022,35.583168,24.302996,59.886162,11.280172,120.810913,-145.169968,-24.359154,265.980865,NaN
4665,93.30,27.930841,20.069925,48.000767,7.860917,24.770229,5.570041,30.340280,19.200237,24.641913,...,-53.854199,35.666756,24.337494,60.004250,11.329263,120.908920,-145.140564,-24.231743,266.049469,NaN
4666,93.32,28.004835,20.072241,48.077076,7.932594,24.837492,5.581850,30.419352,19.255692,24.724546,...,-53.795460,35.756123,24.348608,60.104729,11.407514,120.992508,-145.106064,-24.113655,266.098572,NaN
4667,93.34,28.043264,20.094236,48.137501,7.949027,24.913395,5.611948,30.525353,19.301496,24.791809,...,-53.766090,35.802239,24.375002,60.177239,11.427235,121.082825,-145.081055,-23.998331,266.163879,NaN


In [16]:
importedDataLog

,ID,A1- ECG baseline start,B1- ECG baseline end,C1- VR baseline start,D1- VR baseline end,E1- VR training start,F1- VR training end,G1- Food selection 1 (med-portion) start,H1- Food selection 1 (med-portion) end,I1- Food selection 2 (large-portion) start,J1- Food selection 2 (large-portion) end
0,2,0.0,1.574870e+12,1.574870e+12,1.574871e+12,1.574871e+12,1.574871e+12,1.574871e+12,1.574871e+12,1.574871e+12,1.574872e+12
1,3,0.0,1.575649e+12,1.575649e+12,1.575649e+12,NaN,NaN,1.575649e+12,1.575649e+12,1.575649e+12,1.575650e+12
2,5,0.0,1.576772e+12,1.576772e+12,1.576772e+12,1.576772e+12,1.576773e+12,1.576773e+12,1.576773e+12,1.576773e+12,1.576774e+12
3,877,0.0,1.579023e+12,1.579023e+12,1.579023e+12,1.579023e+12,1.579024e+12,1.579024e+12,1.579024e+12,1.579024e+12,1.579024e+12
4,820,0.0,1.578332e+12,1.578332e+12,1.578332e+12,1.578333e+12,1.578333e+12,1.578333e+12,1.578333e+12,1.578333e+12,1.578333e+12
5,937,0.0,1.580494e+12,1.580494e+12,1.580494e+12,1.580495e+12,1.580495e+12,1.580495e+12,1.580495e+12,1.580495e+12,1.580495e+12
6,942,0.0,1.581531e+12,1.581531e+12,1.581531e+12,1.581531e+12,1.581531e+12,1.581531e+12,1.581531e+12,1.581531e+12,1.581532e+12
7,793,0.0,1.581447e+12,1.581447e+12,1.581447e+12,1.581447e+12,1.581447e+12,1.581447e+12,1.581448e+12,1.581448e+12,1.581448e+12
8,961,0.0,1.581964e+12,1.581964e+12,1.581964e+12,1.581965e+12,1.581965e+12,1.581965e+12,1.581965e+12,1.581965e+12,1.581965e+12
9,963,0.0,1.582044e+12,1.582044e+12,1.582045e+12,1.582045e+12,1.582045e+12,1.582045e+12,1.582045e+12,1.582045e+12,1.582045e+12


In [28]:
importedData2['1'][0]

1583861706.02

NameError: name 'average' is not defined